## The following modules will be used:
* BeautifulSoup - used to crawle the website
* pandas used to arrange data in tables

In [1]:
# Installation of the necessary libraries
!pip install requests --quiet
!pip install beautifulsoup4 --quiet
!pip install pymongo --quiet
!pip install urllib3 --quiet

In [2]:
import csv
import requests
import pandas as pd
import urllib
import time

from bs4 import BeautifulSoup
from pymongo import MongoClient
from IPython.display import display

In [3]:
%%bash
sudo apt install -y mongodb > log
service mongodb start

 * Starting database mongodb
   ...done.




debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 8.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [4]:
# Wait for the instance to start
time.sleep(5)

In [5]:
%%bash
ps -ef | grep mongo

mongodb      545       1 13 13:20 ?        00:00:00 /usr/bin/mongod --config /etc/mongodb.conf
root         589     587  0 13:20 ?        00:00:00 grep mongo


## Instantiate mongodb

In [6]:
mongoClient = MongoClient()
# Get the atlas mongodb connection string
# mongo_uri = "mongodb://vineethr238:" + urllib.parse.quote("vini8686", safe="") + "@cluster0.yqicrpo.mongodb.net/?retryWrites=true&w=majority"
# client = MongoClient(mongo_uri)
client = MongoClient()
client.list_database_names()
# db = client.stocknews
# store = db.stocks

# db = client.stocknews.stocks

['admin', 'local']

In [7]:
DATABASE_URI = "mongodb://localhost:27017"
DATABASE = "stocknews"
NEWS_COLLECTION = "stocks"
db = client[DATABASE]

# Check existence of collection
if "stocks" not in db.list_collection_names():
  db.create_collection(NEWS_COLLECTION)

## Test the Base Url

In [8]:
BASE_URL = 'https://finance.yahoo.com' 

## Utility Methods

In [9]:
def get_page(url):
    """Download a webpage and return a beautiful soup doc"""
    response = requests.get(url)
    if not response.ok:
        print('Status code:', response.status_code)
        raise Exception('Failed to load page {}'.format(url))
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc


def get_news_tags(doc):
    """Get the list of tags containing news information"""
    news_class = "Ov(h) Pend(44px) Pstart(25px)" ## class name of div tag 
    news_list  = doc.find_all('div', {'class': news_class})
    return news_list


def parse_news(news_tag):
    """Get the news data point and return dictionary"""
    news_source = news_tag.find('div').text #source
    news_headline = news_tag.find('a').text #heading
    news_url = news_tag.find('a')['href'] #link
    news_content = news_tag.find('p').text #content
    news_image = news_tag.findParent().find('img')['src'] #thumb image
    return { 'source' : news_source,
            'headline' : news_headline,
            'url' : BASE_URL + news_url,
            'content' : news_content,
            'image' : news_image
           }

def save_data_to_mongo(dataframe):
    """Takes a dataframe, converts it to dictionary and saves to mongo db"""
    #     data = dataframe.reset_index(drop=True)
    #     data_dictionary = data.to_dict(data)
    db.stocks.insert_many(dataframe.to_dict(orient="records"))

In [10]:


def scrape_yahoo_news(url, path=None):
    """Get the yahoo finance market news and write them to CSV file """
    if path is None:
        path = 'stock-market-news.csv'
    print('Requesting html page')
    doc = get_page(url)
    print('Extracting news tags')
    news_list = get_news_tags(doc)
    print('Parsing news tags')
    news_data = [parse_news(news_tag) for news_tag in news_list]
    print('Save the data to a CSV')
    news_df = pd.DataFrame(news_data)
    news_df.to_csv(path, index=None)
    display(news_df.head())
    save_data_to_mongo(news_df)
    print("Successful pipeline execution")


if __name__ == "__main__" :
    
    YAHOO_NEWS_URL = BASE_URL+'/news/'
    news_df = scrape_yahoo_news(YAHOO_NEWS_URL)

print("Processing Done")

Requesting html page
Extracting news tags
Parsing news tags
Save the data to a CSV


,source,headline,url,content,image
0,Bloomberg,Ghana Catches Investors by Surprise as It Susp...,https://finance.yahoo.com/news/ghana-catches-i...,(Bloomberg) -- Ghana on Monday suspended inter...,https://s.yimg.com/uu/api/res/1.2/jvzUnPAZnSsg...
1,Yahoo Finance,Yuga Labs names Activision Blizzard exec Danie...,https://finance.yahoo.com/news/yuga-labs-names...,Activision Blizzard exec Daniel Alegre is join...,https://s.yimg.com/uu/api/res/1.2/iNwOmAoFGM7f...
2,Reuters,Special Report-Binance's books are a black box...,https://finance.yahoo.com/news/special-report-...,"The world's biggest crypto exchange, Binance, ...",https://s.yimg.com/uu/api/res/1.2/aCIW0vhV_waD...
3,Reuters,Morgan Stanley appoints two co-heads of Italia...,https://finance.yahoo.com/news/morgan-stanley-...,Morgan Stanley has named Emilio Greco and Nico...,https://s.yimg.com/uu/api/res/1.2/95CAMwHV6Dzc...


Successful pipeline execution
Processing Done


In [11]:
# confirm data storage
db.stocks.count
db.list_collection_names()

['stocks']

In [12]:
for item in db.stocks.find():
  print(item)

{'_id': ObjectId('63a06521d33f9c2e9405b1ca'), 'source': 'Bloomberg', 'headline': 'Ghana Catches Investors by Surprise as It Suspends Debt Payments', 'url': 'https://finance.yahoo.com/news/ghana-catches-investors-surprise-suspends-130926858.html', 'content': "(Bloomberg) -- Ghana on Monday suspended interest payments on its external debt, catching bondholders by surprise ahead of restructuring talks aimed at unlocking an International Monetary Fund bailout.Most Read from BloombergMusk Polls Twitter to Quit as Chief, Voters Leaning Toward YesTruth Is Reasserting Itself Over Trump’s LiesTrump Gets a 45-Day Extension to Provide Details on His Wealth to VotersHarry and Meghan Aren't Doing Themselves Any FavorsMessi May Not Be Soccer’s GOAT for LongThe We", 'image': 'https://s.yimg.com/uu/api/res/1.2/jvzUnPAZnSsgH82UGqh1_g--~B/Zmk9c3RyaW07aD0xMjM7cT04MDt3PTIyMDthcHBpZD15dGFjaHlvbg--/https://media.zenfs.com/en/bloomberg_markets_842/dd19686f0a4ace9c64e4b6661b43a63e.cf.jpg'}
{'_id': ObjectId('6